## Preprocesamiento de los datos.

Para lograr implementar cualquier modelo con éxito y resolver un problema, es importante realizar un preprocesamiento a los datos que se vayan a usar. Este proceso siempre se debe emplear porque los datos pueden estar sucios, contener ruido, datos faltantes, caracteres especiales, signos de puntuación, entre otras, los cuales no permitirán que el modelo sea entrenado correctamente. Por tal motivo, a continuación se efectúa el proceso de verificación, limpieza, transformación y división del conjunto para entrenamiento y pruebas

In [36]:
#Cargar librerias
import pandas as pd
import numpy as np
import nltk 
import spacy
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelBinarizer
from nltk.corpus import stopwords
from bs4 import BeautifulSoup
import re,string,unicodedata

from nltk.tokenize import word_tokenize,sent_tokenize
from nltk.tokenize.toktok import ToktokTokenizer

import warnings
warnings.filterwarnings('ignore')


El conjunto de datos que utilizara para el proceso de análisis de sentimientos en reseñas de películas, está conformado por 50 mil datos. El conjunto de datos está constituido por dos columnas (review, sentiment). Para cargar los datos, se utiliza la librería Pandas y se muestra los 10 primeros datos que contiene el datset. La columna review contiene información con datos sucios y ruidos. 

In [24]:
#Muestra los 10 primeros review
data=pd.read_csv('IMDB Dataset.csv')
print(data.shape)
data.head(10)

(50000, 2)


,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive
5,"Probably my all-time favorite movie, a story o...",positive
6,I sure would like to see a resurrection of a u...,positive
7,"This show was an amazing, fresh & innovative i...",negative
8,Encouraged by the positive comments about this...,negative
9,If you like original gut wrenching laughter yo...,positive


In [25]:
#Imprime el review que se encuentra en la posición 4 
data['review'][4]

'Petter Mattei\'s "Love in the Time of Money" is a visually stunning film to watch. Mr. Mattei offers us a vivid portrait about human relations. This is a movie that seems to be telling us what money, power and success do to people in the different situations we encounter. <br /><br />This being a variation on the Arthur Schnitzler\'s play about the same theme, the director transfers the action to the present time New York where all these different characters meet and connect. Each one is connected in one way, or another to the next person, but no one seems to know the previous point of contact. Stylishly, the film has a sophisticated luxurious look. We are taken to see how these people live and the world they live in their own habitat.<br /><br />The only thing one gets out of all these souls in the picture is the different stages of loneliness each one inhabits. A big city is not exactly the best place in which human relations find sincere fulfillment, as one discerns is the case wit

### Verificar y limpiar de datos

Una vez cargados los datos se verifica que el dataset no contenga valores nulos. Y para esto, se utiliza la función isnull para la comprobación. Esta función devolverá "True" si encuentra componentes faltantes y "False" para los componentes que no faltan. Finalmente, se contará todos los valores "True" y como se puede observar, el conteo dio 0, es decir, no existe valores nulos en todo el dataset. 

In [26]:
#Se verifica si existe valores nullos en el dataset
data.isnull().sum()

review       0
sentiment    0
dtype: int64

En las siguientes líneas de código se realiza el proceso de limpieza de los datos. Se desarrolla una función para eliminar cualquier elemento HTML que exista en los review. Otra función para eliminar expresiones regulares, como lo son corchetes, llaves, asteriscos, comillas, entre otros. También se elimina texto ruidoso y finalmente, la eliminación de caracteres especiales. 

In [27]:
#Eliminar etiquetas de HTML 
def strip_html(text):
    soup = BeautifulSoup(text, "html.parser")
    return soup.get_text()

#Eliminar algunas expresiones
def remove_between_square_brackets(text):
    return re.sub('\[[^]]*\]', '', text)

#Elimina texto ruidoso
def denoise_text(text):
    text = strip_html(text)
    text = remove_between_square_brackets(text)
    return text

#Aplicar la función denoise_text en la columna de review
data['review']=data['review'].apply(denoise_text)

#---------------------------------------------

#Remueve caracteres especiales
def remove_special_characters(text, remove_digits=True):
    pattern=r'[^a-zA-z0-9\s]'
    text=re.sub(pattern,'',text)
    return text

#Aplicar la función remove_special_characters en la columna review
data['review']=data['review'].apply(remove_special_characters)

In [28]:
#Se verifica que los datos esten limpios
data['review'][4]

'Petter Matteis Love in the Time of Money is a visually stunning film to watch Mr Mattei offers us a vivid portrait about human relations This is a movie that seems to be telling us what money power and success do to people in the different situations we encounter This being a variation on the Arthur Schnitzlers play about the same theme the director transfers the action to the present time New York where all these different characters meet and connect Each one is connected in one way or another to the next person but no one seems to know the previous point of contact Stylishly the film has a sophisticated luxurious look We are taken to see how these people live and the world they live in their own habitatThe only thing one gets out of all these souls in the picture is the different stages of loneliness each one inhabits A big city is not exactly the best place in which human relations find sincere fulfillment as one discerns is the case with most of the people we encounterThe acting i

In [29]:
data.head(10)

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production The filming tech...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically theres a family where a little boy J...,negative
4,Petter Matteis Love in the Time of Money is a ...,positive
5,Probably my alltime favorite movie a story of ...,positive
6,I sure would like to see a resurrection of a u...,positive
7,This show was an amazing fresh innovative ide...,negative
8,Encouraged by the positive comments about this...,negative
9,If you like original gut wrenching laughter yo...,positive


### Tokenizar

La tokenización sirve para encontrar y comprender patrones en el texto, el cual es útil para realizar el proceso de análisis de sentimiento. Este proceso es base para derivación y lematización del texto. Por tal motivo, en la siguiente línea de código se emplea el token y se configura las palabras vacías en el idioma en inglés.

In [30]:
#Tokenización de texto
tokenizer=ToktokTokenizer()
#Configuración de palabras vacías en inglés
stopword_list=nltk.corpus.stopwords.words('english')

### Stemming 

La implementación del Stemming se encarga de reducir las palabras a su raíz. Y es útil para asignar varias palabras a una palabra base en específico, esto no solo se hace con palabras, sino también con oraciones. Por tal motivo, en la siguiente línea de código se emplea Steamming en el texto.  

In [31]:
#Stemming del texto
def simple_stemmer(text):
    ps=nltk.porter.PorterStemmer()
    text= ' '.join([ps.stem(word) for word in text.split()])
    return text
#Aplicar la función en la columna review 
data['review']=data['review'].apply(simple_stemmer) 

### Eliminar palabras varias 

Con el proceso de tokenización, se desarrolla una función que elimina todas las palabras vacias y se aplica el metodo en la columna review. 

In [32]:
#establecer palabras vacías en inglés
stop=set(stopwords.words('english'))
print(stop)

#eliminando las palabras vacías 
def remove_stopwords(text, is_lower_case=False):
    tokens = tokenizer.tokenize(text)
    tokens = [token.strip() for token in tokens]
    
    if is_lower_case:
        filtered_tokens = [token for token in tokens if token not in stopword_list]
    else:
        filtered_tokens = [token for token in tokens if token.lower() not in stopword_list]
    filtered_text = ' '.join(filtered_tokens)    
    return filtered_text

#Aplicar la función 
data['review']=data['review'].apply(remove_stopwords)

{'any', 'their', 'don', 'ma', 'will', 'hers', 'll', "hasn't", 'against', 'this', "you'd", "won't", 'being', 'aren', 'yourselves', 'too', "isn't", 'then', "doesn't", 'both', 'can', 'is', 'they', 'there', 'haven', 'not', "it's", 're', 'through', 'do', "should've", 'ain', 'shouldn', 'has', 'other', 'of', 'her', 'weren', "you've", 'when', 'further', 'and', 'needn', "needn't", 'as', 'for', 'these', 'all', 'd', "that'll", "couldn't", 'here', 'down', 'if', "shan't", 'on', 'out', 'whom', "haven't", 'mustn', 'you', 'just', "mustn't", 'doesn', 'are', 'shan', "wouldn't", 'after', 'wouldn', 'because', 'no', 'hadn', "didn't", 'few', 'should', 'them', 'from', 'again', 'that', 'did', "hadn't", 'themselves', 'how', 'the', 'been', 'she', 've', 'each', 'hasn', 'couldn', 'or', 'm', "aren't", 'it', 'isn', 'off', 'him', 'your', 'be', 'why', 'ourselves', 'mightn', 'was', 'himself', 'to', 'below', 'during', "she's", 'while', 'y', 'before', 'so', "weren't", 'only', 'yours', 'some', 'my', 'which', 'i', 'theirs

### Normalización 

Una vez terminado el proceso de limpieza de los datos, se emplea la normalización, es decir, estandariza todos los datos a un solo nive (palabras en minusculas) 

In [33]:
#nornalizar los reviews 
norm_reviews=data.review
norm_reviews[4] 

'petter mattei love time money visual stun film watch mr mattei offer us vivid portrait human relat thi movi seem tell us money power success peopl differ situat encount thi variat arthur schnitzler play theme director transfer action present time new york differ charact meet connect one connect one way anoth next person one seem know previou point contact stylishli film ha sophist luxuri look taken see peopl live world live habitatth onli thing one get soul pictur differ stage loneli one inhabit big citi exactli best place human relat find sincer fulfil one discern case peopl encounterth act good mr mattei direct steve buscemi rosario dawson carol kane michael imperioli adrian grenier rest talent cast make charact come alivew wish mr mattei good luck await anxious hi next work'

### Modelo Bags of words


Para realizar análisis de sentimiento se necesita crear una bolsa de palabras. El modelo Bag of Words es utilizado para representar los datos de prueba. Este método es empleado para PLN y IR. Para implementar este modelo se genera un vector de bolsas de palabras y después se hace un ajuste de los datos. 

In [41]:
#Contar vectorizador para bolsa de palabras
cv=CountVectorizer(min_df=0,max_df=1,binary=False,ngram_range=(1,3))

#Transformar datos
cv_reviews=cv.fit_transform(norm_reviews)


### Transformación del etiquetado a datos binarios 

Una vez ya procesado los datos pertenecientes a la columna review, ahora solo falta la columna sentiment. En el caso de este dataset, la columna sentiment únicamente tiene dos valores: positive, negative. Y como tal, estos valores se los pueden convertir en números binarios, en donde el 0 representa a los negativos y el 1 a los positivos. La función LabelBinariser hace este proceso de binarización. 

In [42]:
#Función para convertir en binarios
lb=LabelBinarizer()

#transformación de los datos 
data['sentiment']=lb.fit_transform(data['sentiment'])
sentiment_data = data['sentiment']
print(sentiment_data.shape)
#Datos Limpios
data.head(10)

(50000,)


,review,sentiment
0,one review ha mention watch 1 oz episod youll ...,1
1,wonder littl product film techniqu veri unassu...,1
2,thought thi wa wonder way spend time hot summe...,1
3,basic famili littl boy jake think zombi hi clo...,0
4,petter mattei love time money visual stun film...,1
5,probabl alltim favorit movi stori selfless sac...,1
6,sure would like see resurrect date seahunt ser...,1
7,thi show wa amaz fresh innov idea 70 first air...,0
8,encourag posit comment thi film wa look forwar...,0
9,like origin gut wrench laughter like thi movi ...,1


In [43]:
ruta = "review_limpios.csv"
#data.to_csv(ruta)

Ya terminada la limpieza y transformación de los datos, se procede a dividir el conjunto en datos de entrenamiento y datos para las pruebas. El conjunto de entrenamiento contiene 35 000 revisiones (70 % de los datos) destinadas al entrenamiento y el conjunto de prueba tiene 15 000 revisiones (30 % de los datos) para probar el clasificador.

### Dividir el conjunto

In [44]:
#Dividir el dataset en datos de entrenamiento y prueba 
X_train_reviews = cv_reviews[:35000]
Y_train_sentiments = sentiment_data[:35000]

X_test_reviews = cv_reviews[35000:]
Y_test_sentiments = sentiment_data[35000:]


El proceso de clasificación de características no se lo realiza porque el dataset solamente tiene una sola característica.